# Portfolio Strategy Analysis: CPPI vs Buy & Hold

## Research Question
Does a **CPPI** outperform a **Buy & Hold** benchmark over the sample period ***[2000-2023]*** in terms of risk-adjusted performance?

## Strategy Chosen
- Strategy type: **CPPI**
- Assets analyzed: **S&P 500, S&P U.S. Treasury Bill Index**
- Rebalancing frequency: **[daily/weekly/monthly]**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-v0_8")

import yfinance as yf
from datetime import datetime

## Data Import
We import price data for the following tickers:

- **S&P 500**
- **S&P U.S. Treasury Bill Index**  
- Period: **[01/01/2000 – 31/12/2023]**

In [ ]:
tickers = ["SPY", "^SPBDUBIT"]   # example
start = "2000-01-01"
end = "2023-12-31"

prices = yf.download(tickers, start=start, end=end)["Adj Close"]
prices.head()

## Raw Price Series
We plot the raw adjusted close prices to visually inspect data quality.

In [ ]:
prices.plot(figsize=(12,5), title="Raw Price Series")
plt.show()

## Data Cleaning
- Handle missing values  
- Check for price jumps  
- Convert prices to log or simple returns

In [ ]:
# Missing value handling
prices = prices.dropna()

# Compute returns
returns = prices.pct_change().dropna()
returns.head()

## Return Properties
We inspect return behavior: distribution, volatility, etc.

In [ ]:
returns.plot(figsize=(12,4), title="Daily Returns")
plt.show()

# Rolling volatility example
roll_vol = returns.rolling(21).std()
roll_vol.plot(figsize=(12,4), title="21-Day Rolling Volatility")
plt.show()

## Buy & Hold Benchmark
We construct a static allocation for comparison.

- Weights: **[e.g., 60% SPY, 40% SPBDUBI]**

In [ ]:
bh_weights = np.array([0.6, 0.4])
bh_returns = (returns * bh_weights).sum(axis=1)
bh_cum = (1 + bh_returns).cumprod()

bh_cum.plot(figsize=(12,4), title="Buy & Hold Cumulative Return")
plt.show()

## The strategy

In [ ]:
# === PARAMETERS ===
m = 3
floor = 0.8  # 80% capital floor
portfolio_value = 1.0

cppi_vals = []
weight_risky = []

for t in range(len(returns)):
    cushion = portfolio_value - floor
    w = m * cushion
    w = np.clip(w, 0, 1)

    r = w * returns["SPY"].iloc[t] + (1-w) * 0  # cash return = 0
    portfolio_value *= (1 + r)

    cppi_vals.append(portfolio_value)
    weight_risky.append(w)

strategy_cum = pd.Series(cppi_vals, index=returns.index)

## Performance Comparison

In [ ]:
ax = strategy_cum.plot(figsize=(12,5), label="Strategy", color="blue")
bh_cum.plot(ax=ax, label="Buy & Hold", color="black")
plt.title("Strategy vs Buy & Hold")
plt.legend()
plt.show()

## Key Performance Metrics
We compute:

- Annualized return  
- Annualized volatility  
- Sharpe ratio  
- Sortino ratio  
- Max drawdown  
- Calmar ratio  

In [ ]:
def ann_return(r):
    return (1+r).prod()**(252/len(r)) - 1

def ann_vol(r):
    return r.std() * np.sqrt(252)

def sharpe(r, rf=0):
    return (ann_return(r)-rf) / ann_vol(r)

def max_drawdown(cum):
    running_max = cum.cummax()
    dd = cum / running_max - 1
    return dd.min()

metrics = pd.DataFrame({
    "Buy & Hold": [
        ann_return(bh_returns),
        ann_vol(bh_returns),
        sharpe(bh_returns),
        max_drawdown(bh_cum)
    ],
    "Strategy": [
        ann_return(strategy_returns),
        ann_vol(strategy_returns),
        sharpe(strategy_returns),
        max_drawdown(strategy_cum)
    ]
}, index=["Ann Return","Ann Vol","Sharpe","Max Drawdown"])

metrics

## Drawdowns

In [ ]:
# Drawdown plots
bh_dd = bh_cum / bh_cum.cummax() - 1
st_dd = strategy_cum / strategy_cum.cummax() - 1

plt.figure(figsize=(12,5))
bh_dd.plot(label="Buy & Hold Drawdown")
st_dd.plot(label="Strategy Drawdown")
plt.legend()
plt.title("Drawdowns")
plt.show()

## Sensitivity Analysis
Try different parameters:  
- Different target vol levels  
- Different CPPI multipliers  
- Different lookback windows

Show their effect on:  
- Return  
- Sharpe  
- Drawdown  

In [ ]:
# Example loop for sensitivity:
vol_targets = [0.08, 0.10, 0.12, 0.15]
results = []

for tv in vol_targets:
    w = (tv / annual_vol).clip(0, 3)    
    r = w.shift(1)*returns["SPY"]
    results.append([tv, ann_return(r), sharpe(r), max_drawdown((1+r).cumprod())])

pd.DataFrame(results, columns=["Target Vol","Ann Ret","Sharpe","MaxDD"])


## Interpretation
Discuss:

- Why the strategy performed differently from Buy & Hold  
- Behavior in volatility spikes  
- Performance during crashes (COVID, 2022 inflation, etc.)  
- Leverage dynamics  
- Advantages and drawbacks

This is where human interpretation matters.


# Conclusion
Answer your research question:

**Does the strategy outperform Buy & Hold? Under what conditions?**  
Mention limitations and possible improvements.

This section must clearly include human reasoning.
